# 算法题 python实现 深入理解

## 704.二分查找

给定一个 n 个元素有序的（升序）整型数组 nums 和一个目标值 target  ，写一个函数搜索 nums 中的 target，如果目标值存在返回下标，否则返回 -1

Case1：
```
输入: nums = [-1,0,3,5,9,12], target = 9     
输出: 4       
解释: 9 出现在 nums 中并且下标为 4  
```

Case2:
```
输入: nums = [-1,0,3,5,9,12], target = 2     
输出: -1        
解释: 2 不存在 nums 中因此返回 -1 
```


提示：

- 你可以假设 nums 中的所有元素是不重复的。
- n 将在 [1, 10000]之间。
- nums 的每个元素都将在 [-9999, 9999]之间。

**解**
1. 用enumerate似乎太偷懒了
2. 就用正常的二分法
3. 边界处需要注意开闭区间

In [24]:
def search(nums: list[int], target: int) -> int:
    """二分法 默认左右闭区间"""
    left = 0
    right = len(nums) - 1
    while left <= right:
        middle = left + (right - left) // 2

        if nums[middle] > target: # 左区间
            right = middle - 1
        elif nums[middle] < target: # 右区间
            left = middle + 1
        else:
            return middle
    return -1

In [25]:
result = search(
    nums = [-1,0,3,5,9,12],
    target=9
)
result

4

In [28]:
"""
更偷懒的方法
"""
nums = [-1,0,3,5,9,12]
target=9
nums.index(target)

4

## 27. 移除元素

给你一个数组 nums 和一个值 val，你需要 原地 移除所有数值等于 val 的元素，并返回移除后数组的新长度。

不要使用额外的数组空间，你必须仅使用 O(1) 额外空间并原地修改输入数组。

元素的顺序可以改变。你不需要考虑数组中超出新长度后面的元素。

示例 1: 给定 nums = [3,2,2,3], val = 3, 函数应该返回新的长度 2, 并且 nums 中的前两个元素均为 2。 你不需要考虑数组中超出新长度后面的元素。

示例 2: 给定 nums = [0,1,2,2,3,0,4,2], val = 2, 函数应该返回新的长度 5, 并且 nums 中的前五个元素为 0, 1, 3, 0, 4。

你不需要考虑数组中超出新长度后面的元素。

**解答**
1. 很偷懒的方法, 直接list.remove(target), 但可能不符合时间复杂度的要求
2. 快慢指针，快指针用来比较、慢指针用来记录新的数组

In [29]:
nums = [0,1,2,3,3,0,4,2]
target = 2

In [30]:
nums.remove(target)
nums

[0, 1, 3, 3, 0, 4, 2]

In [31]:
def removeElement(nums: list[int], val: int) -> int:
    fast = 0
    slow = 0
    size = len(nums)
    while fast < size:
        if nums[fast] != val:
            nums[slow] = nums[fast]
            slow += 1
        fast += 1
    return slow

In [32]:
removeElement(nums, target)

6

## 977. 有序数组的平方

给你一个按 非递减顺序 排序的整数数组 nums，返回 每个数字的平方 组成的新数组，要求也按 非递减顺序 排序。

```
输入：nums = [-4,-1,0,3,10]
输出：[0,1,9,16,100]
解释：平方后，数组变为 [16,1,0,9,100]，排序后，数组变为 [0,1,9,16,100]
示例 2：

输入：nums = [-7,-3,2,3,11]
输出：[4,9,9,49,121]
```

**解答**
1. 直觉是暴力法，平方完，然后排序
2. 解析一下就是，数组是有序的（包含负数），平方后最大值不是在最左边就是在最右边，不可能在中间
3. 定义双指针, 从两边从中间数

In [33]:
def sortedSquares(nums: list[int]) -> list[int]:
    left = 0
    right = len(nums) - 1
    i = len(nums) - 1
    res = [float('inf')] * len(nums)
    while left <= right:
        if nums[left] ** 2 < nums[right] ** 2:
            res[i] = nums[right] ** 2
            right -= 1
        else:
            res[i] = nums[left] ** 2
            left += 1
        i -= 1
    return res

In [34]:
nums =  [-4,-1,0,3,10]
sortedSquares(nums)

[0, 1, 9, 16, 100]